In [1]:
# Same as the ManyToOne example, except OneToOne is intended to model
# is-a (inheritance) relationships, and only one child per relation is allowed.

In [2]:
# Clean up in case of re-running
OneToOneChildProtect.objects.all().delete()
OneToOneParent.objects.all().delete()
OneToOneChildCascade.objects.all().delete()
OneToOneChildNull.objects.all().delete();

In [3]:
# Populate a parent with all 3 kinds of children.
parent = OneToOneParent.objects.create()

OneToOneChildCascade.objects.create(parent=parent)
OneToOneChildNull.objects.create(parent=parent)
OneToOneChildProtect.objects.create(parent=parent)

[p.print_relations() for p in OneToOneParent.objects.all()];

OneToOneParent id: 1 has cascade child OneToOneChildCascade 2
OneToOneParent id: 1 has null child OneToOneChildNull 3
OneToOneParent id: 1 has protect child OneToOneChildProtect 4


In [4]:
# Deletion behavior of the child does not depend on the type.
parent.child_cascade.delete()
parent.child_null.delete()
parent.child_protect.delete()

[p.print_relations() for p in OneToOneParent.objects.all()];

Delete called on OneToOneChildCascade 2
Delete called on OneToOneChildNull 3
Delete called on OneToOneChildProtect 4
OneToOneParent id: 1 has no cascade child
OneToOneParent id: 1 has no null child
OneToOneParent id: 1 has no protect child


In [5]:
# But when deleting the parent, the behavior depends on on_delete in the child OneToOneField to the parent
def get_ids_parents(ModelClass):
    return [t for t in ModelClass.objects.values_list("id", "parent", named=True)]

OneToOneChildCascade.objects.create(parent=parent)
OneToOneChildNull.objects.create(parent=parent)

print('Existing children')
print(f'OneToOneChildCascade: {get_ids_parents(OneToOneChildCascade)}')
print(f'OneToOneChildNull: {get_ids_parents(OneToOneChildNull)}')

print('\nDeleting the parent of these children\n')
parent.delete()

# Here be the dragon!
print(f'OneToOneChildCascade: {get_ids_parents(OneToOneChildCascade)} <--Got deleted with the parent!')
print(f'OneToOneChildNull: {get_ids_parents(OneToOneChildNull)} <-- Orphan :_( ');

Existing children
OneToOneChildCascade: [Row(id=5, parent=1)]
OneToOneChildNull: [Row(id=6, parent=1)]

Deleting the parent of these children

Delete called on OneToOneParent 1
OneToOneChildCascade: [] <--Got deleted with the parent!
OneToOneChildNull: [Row(id=6, parent=None)] <-- Orphan :_( 


In [6]:
# Now the final case - Try to delete a parent protected by a child
from django.db.models import ProtectedError

parent = OneToOneParent.objects.create()
OneToOneChildProtect.objects.create(parent=parent)

try:
    parent.delete()
    print('\nThe parent was deleted!\n')
except ProtectedError:
    print('\nThe parent could not be deleted!\n')
    parent.print_relations()

Delete called on OneToOneParent 7

The parent could not be deleted!

OneToOneParent id: 7 has no cascade child
OneToOneParent id: 7 has no null child
OneToOneParent id: 7 has protect child OneToOneChildProtect 8


In [7]:
# The child is protecting the parent as long as it exists.  One use of this is to enforce clean-up,
# there can be no orhpaned child with a protected parent.
parent.child_protect.delete()
parent.delete();

Delete called on OneToOneChildProtect 8
Delete called on OneToOneParent 7
